In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from scipy.optimize import curve_fit
import sep
plt.style.use('seaborn-white')

In [2]:
#read in and separate the files, then extract the data
fits_files = [f for f in os.listdir('Nickle/fits_files') if f.endswith('.fits')]

bias_files = fits_files[0:10]
bias_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in bias_files])

domeflat_B_files = fits_files[10:15]
domeflat_V_files = fits_files[15:20]
domeflat_R_files = fits_files[20:25]
domeflat_B_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in domeflat_B_files])
domeflat_V_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in domeflat_V_files])
domeflat_R_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in domeflat_R_files])

cal_B_files = fits_files[26:31]
cal_V_files = fits_files[31:36]
cal_R_files = fits_files[36:41]
cal_B_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in cal_B_files])
cal_V_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in cal_V_files])
cal_R_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in cal_R_files])

M67_B_files = fits_files[41:46]
M67_V_files = fits_files[46:51]
M67_R_files = fits_files[51:56]
M67_B_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in M67_B_files])
M67_V_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in M67_V_files])
M67_R_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in M67_R_files])

M3_B_files = fits_files[97:100]
M3_V_files = fits_files[100:103]
M3_R_files = fits_files[103:108]
M3_B_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in M3_B_files])
M3_V_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in M3_V_files])
M3_R_data = np.array([
    fits.getdata(os.path.join('Nickle/fits_files', file))
    for file in M3_R_files])

In [3]:
#create a 3d array made up of stacked 2d image arrays. make a new 2d array where every pixel count
#is the median of the pixel counts across all images
bias_pixels = np.empty([1024, 1056, 10])
bias_master = np.empty([1024, 1056])
for i in range(0,1024):
    for j in range(0,1056):
        bias_pixels[i][j] = [bias_data[0][i][j], bias_data[1][i][j], bias_data[2][i][j],
            bias_data[3][i][j], bias_data[4][i][j], bias_data[5][i][j],
            bias_data[6][i][j], bias_data[7][i][j], bias_data[8][i][j],
            bias_data[9][i][j]]
        bias_master[i][j] = np.median(bias_pixels[i][j])
#plt.imshow(bias_master, vmax=1200)

In [4]:
domeflat_B_pixels = np.empty([1024, 1056, len(domeflat_B_data)])
domeflat_B_master = np.empty([1024, 1056])
domeflat_V_pixels = np.empty([1024, 1056, len(domeflat_V_data)])
domeflat_V_master = np.empty([1024, 1056])
domeflat_R_pixels = np.empty([1024, 1056, len(domeflat_R_data)])
domeflat_R_master = np.empty([1024, 1056])
for i in range(0, 1024):
    for j in range(0, 1056):
        domeflat_B_pixels[i][j] = [domeflat_B_data[0][i][j], domeflat_B_data[1][i][j],
            domeflat_B_data[2][i][j], domeflat_B_data[3][i][j],
            domeflat_B_data[4][i][j]]
        domeflat_B_master[i][j] = np.median(domeflat_B_pixels[i][j])
        
        domeflat_V_pixels[i][j] = [domeflat_V_data[0][i][j], domeflat_V_data[1][i][j],
            domeflat_V_data[2][i][j], domeflat_V_data[3][i][j],
            domeflat_V_data[4][i][j]]
        domeflat_V_master[i][j] = np.median(domeflat_V_pixels[i][j])
        
        domeflat_R_pixels[i][j] = [domeflat_R_data[0][i][j], domeflat_R_data[1][i][j],
            domeflat_R_data[2][i][j], domeflat_R_data[3][i][j],
            domeflat_R_data[4][i][j]]
        domeflat_R_master[i][j] = np.median(domeflat_R_pixels[i][j])

In [5]:
domeflat_B_clean = domeflat_B_master - bias_master
domeflat_V_clean = domeflat_V_master - bias_master
domeflat_R_clean = domeflat_R_master - bias_master

domeflat_B_clean_norm = domeflat_B_clean / np.mean(domeflat_B_clean)
domeflat_V_clean_norm = domeflat_V_clean / np.mean(domeflat_V_clean)
domeflat_R_clean_norm = domeflat_R_clean / np.mean(domeflat_R_clean)

In [6]:
cal_B_clean = cal_B_data - bias_master
cal_V_clean = cal_V_data - bias_master
cal_R_clean = cal_R_data - bias_master
cal_B_hdul = fits.open(os.path.join('Nickle/fits_files', cal_B_files[0]))
cal_V_hdul = fits.open(os.path.join('Nickle/fits_files', cal_V_files[0]))
cal_R_hdul = fits.open(os.path.join('Nickle/fits_files', cal_R_files[0]))
cal_B_exptime = cal_B_hdul[0].header['EXPTIME']
cal_V_exptime = cal_V_hdul[0].header['EXPTIME']
cal_R_exptime = cal_R_hdul[0].header['EXPTIME']
cal_B_persec = cal_B_clean / cal_B_exptime 
cal_V_persec = cal_V_clean / cal_V_exptime 
cal_R_persec = cal_R_clean / cal_R_exptime 

M67_B_clean = M67_B_data - bias_master
M67_V_clean = M67_V_data - bias_master
M67_R_clean = M67_R_data - bias_master
M67_B_hdul = fits.open(os.path.join('Nickle/fits_files', M67_B_files[0]))
M67_V_hdul = fits.open(os.path.join('Nickle/fits_files', M67_V_files[0]))
M67_R_hdul = fits.open(os.path.join('Nickle/fits_files', M67_R_files[0]))
M67_B_exptime = M67_B_hdul[0].header['EXPTIME']
M67_V_exptime = M67_V_hdul[0].header['EXPTIME']
M67_R_exptime = M67_R_hdul[0].header['EXPTIME']
M67_B_persec = M67_B_clean / M67_B_exptime 
M67_V_persec = M67_V_clean / M67_V_exptime 
M67_R_persec = M67_R_clean / M67_R_exptime

M3_B_clean = M3_B_data - bias_master
M3_V_clean = M3_V_data - bias_master
M3_R_clean = M3_R_data - bias_master
M3_B_hdul = fits.open(os.path.join('Nickle/fits_files', M3_B_files[0]))
M3_V_hdul = fits.open(os.path.join('Nickle/fits_files', M3_V_files[0]))
M3_R_hdul = fits.open(os.path.join('Nickle/fits_files', M3_R_files[0]))
M3_B_exptime = M3_B_hdul[0].header['EXPTIME']
M3_V_exptime = M3_V_hdul[0].header['EXPTIME']
M3_R_exptime = M3_R_hdul[0].header['EXPTIME']
M3_B_persec = M3_B_clean / M3_B_exptime 
M3_V_persec = M3_V_clean / M3_V_exptime 
M3_R_persec = M3_R_clean / M3_R_exptime

In [7]:
cal_B_pixels = np.empty([1024, 1056, len(cal_B_persec)])
cal_B_master = np.empty([1024, 1056])
cal_V_pixels = np.empty([1024, 1056, len(cal_V_persec)])
cal_V_master = np.empty([1024, 1056])
cal_R_pixels = np.empty([1024, 1056, len(cal_R_persec)])
cal_R_master = np.empty([1024, 1056])
for i in range(0, 1024):
    for j in range(0, 1056):
        cal_B_pixels[i][j] = [cal_B_persec[0][i][j], cal_B_persec[1][i][j],
            cal_B_persec[2][i][j], cal_B_persec[3][i][j],
            cal_B_persec[4][i][j]]
        cal_B_master[i][j] = np.median(cal_B_pixels[i][j])
        
        cal_V_pixels[i][j] = [cal_V_persec[0][i][j], cal_V_persec[1][i][j],
            cal_V_persec[2][i][j], cal_V_persec[3][i][j],
            cal_V_persec[4][i][j]]
        cal_V_master[i][j] = np.median(cal_V_pixels[i][j])
        
        cal_R_pixels[i][j] = [cal_R_persec[0][i][j], cal_R_persec[1][i][j],
            cal_R_persec[2][i][j], cal_R_persec[3][i][j],
            cal_R_persec[4][i][j]]
        cal_R_master[i][j] = np.median(cal_R_pixels[i][j])

In [8]:
M67_B_pixels = np.empty([1024, 1056, len(M67_B_persec)])
M67_B_master = np.empty([1024, 1056])
M67_V_pixels = np.empty([1024, 1056, len(M67_V_persec)])
M67_V_master = np.empty([1024, 1056])
M67_R_pixels = np.empty([1024, 1056, len(M67_R_persec)])
M67_R_master = np.empty([1024, 1056])
for i in range(0, 1024):
    for j in range(0, 1056):
        M67_B_pixels[i][j] = [M67_B_persec[0][i][j], M67_B_persec[1][i][j],
            M67_B_persec[2][i][j], M67_B_persec[3][i][j],
            M67_B_persec[4][i][j]]
        M67_B_master[i][j] = np.median(M67_B_pixels[i][j])
        
        M67_V_pixels[i][j] = [M67_V_persec[0][i][j], M67_V_persec[1][i][j],
            M67_V_persec[2][i][j], M67_V_persec[3][i][j],
            M67_V_persec[4][i][j]]
        M67_V_master[i][j] = np.median(M67_V_pixels[i][j])
        
        M67_R_pixels[i][j] = [M67_R_persec[0][i][j], M67_R_persec[1][i][j],
            M67_R_persec[2][i][j], M67_R_persec[3][i][j],
            M67_R_persec[4][i][j]]
        M67_R_master[i][j] = np.median(M67_R_pixels[i][j])

In [9]:
M3_B_pixels = np.empty([1024, 1056, len(M3_B_persec)])
M3_B_master = np.empty([1024, 1056])
M3_V_pixels = np.empty([1024, 1056, len(M3_V_persec)])
M3_V_master = np.empty([1024, 1056])
M3_R_pixels = np.empty([1024, 1056, len(M3_R_persec)])
M3_R_master = np.empty([1024, 1056])
for i in range(0, 1024):
    for j in range(0, 1056):
        M3_B_pixels[i][j] = [M3_B_persec[0][i][j], M3_B_persec[1][i][j], M3_B_persec[2][i][j]]
        M3_B_master[i][j] = np.median(M3_B_pixels[i][j])
        
        M3_V_pixels[i][j] = [M3_V_persec[0][i][j], M3_V_persec[1][i][j], M3_V_persec[2][i][j]]
        M3_V_master[i][j] = np.median(M3_V_pixels[i][j])
        
        M3_R_pixels[i][j] = [M3_R_persec[0][i][j], M3_R_persec[1][i][j], M3_R_persec[2][i][j],
                            M3_R_persec[3][i][j], M3_R_persec[4][i][j]]
        M3_R_master[i][j] = np.median(M3_R_pixels[i][j])

In [11]:
cal_B_calibrated = cal_B_master / domeflat_B_clean_norm
cal_V_calibrated = cal_V_master / domeflat_V_clean_norm
cal_R_calibrated = cal_R_master / domeflat_R_clean_norm

M67_B_calibrated = M67_B_master / domeflat_B_clean_norm
M67_V_calibrated = M67_V_master / domeflat_V_clean_norm
M67_R_calibrated = M67_R_master / domeflat_R_clean_norm

M3_B_calibrated = M3_B_master / domeflat_B_clean_norm
M3_V_calibrated = M3_V_master / domeflat_V_clean_norm
M3_R_calibrated = M3_R_master / domeflat_R_clean_norm

<ipython-input-11-ed2bf9061a2c>:1: RuntimeWarning: invalid value encountered in true_divide
  cal_B_calibrated = cal_B_master / domeflat_B_clean_norm
<ipython-input-11-ed2bf9061a2c>:2: RuntimeWarning: invalid value encountered in true_divide
  cal_V_calibrated = cal_V_master / domeflat_V_clean_norm
<ipython-input-11-ed2bf9061a2c>:3: RuntimeWarning: invalid value encountered in true_divide
  cal_R_calibrated = cal_R_master / domeflat_R_clean_norm
<ipython-input-11-ed2bf9061a2c>:5: RuntimeWarning: invalid value encountered in true_divide
  M67_B_calibrated = M67_B_master / domeflat_B_clean_norm
<ipython-input-11-ed2bf9061a2c>:6: RuntimeWarning: invalid value encountered in true_divide
  M67_V_calibrated = M67_V_master / domeflat_V_clean_norm
<ipython-input-11-ed2bf9061a2c>:7: RuntimeWarning: invalid value encountered in true_divide
  M67_R_calibrated = M67_R_master / domeflat_R_clean_norm
<ipython-input-11-ed2bf9061a2c>:9: RuntimeWarning: invalid value encountered in true_divide
  M3_B_c